# Issue 132_2 Notebook Toaletter nära SAT
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_2_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb), version 1 [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb)
* [Issue 132](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132)

Se liknande lösning för Roslagsleden
* nu har vi SAT = wikidata [Q131318799](https://www.wikidata.org/wiki/Q131318799)
* "leden" sitter inte ihop utan varje ö har sitt segment

Output 
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)

In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()
print(timestamp)  # Outputs seconds since Unix epoch


start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


1753851855.180792
Start: 2025-07-30 07:04:15


In [8]:
# -*- coding: utf-8 -*-
"""
Stockholm Archipelago Trail – toaletter nära leden
Omskriven helhet med:
- Sparar specifik fil (2025_07_30_07_21), "latest" och aktuell tidsstämpel
- Fler buffertlager (50 m, 100 m, 200 m) med färgkodning
- Ett enda lager för själva leden (inte ett lager per sektion)
- Separat lager för wheelchair=yes med egen ikon
- Popup med länkar till både OSM och Wheelmap
- About-box högst upp på kartan

OBS: Skriptet anropar Wikidata SPARQL och Overpass API (internet krävs).
"""

# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import os
from datetime import datetime
from collections import defaultdict
import html as _html
from string import Template

import requests
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup

# =====================
# 0. Hjälpfunktioner
# =====================

def save_map_multiple(m: folium.Map):
    """Spara kartan till önskade filnamn.
    - Specifik tidsstämpel: ..._2025_07_30_07_21.html
    - Latest: ..._latest.html
    - Nuvarande tidsstämpel: ..._{YYYY_MM_DD_HH_MM}.html
    """
    os.makedirs("kartor", exist_ok=True)

    # a) Specifik fil som efterfrågats
    specific_path = (
        "kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_21.html"
    )
    m.save(specific_path)

    # b) Latest
    latest_path = (
        "kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_latest.html"
    )
    m.save(latest_path)

    # c) Även nuvarande tidsstämpel för spårbarhet
    timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
    ts_path = f"kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_{timestamp}.html"
    m.save(ts_path)

    print(f"✅ Karta sparad:\n  • {specific_path}\n  • {latest_path}\n  • {ts_path}")


# =================== About-box (Notebook/Edge-safe) =========================
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    """
    Notebook/Edge-säker About-box med string.Template ($-placeholder).
    """
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
  @media (max-width: 640px) { .sat-about { font-size: 11px; min-width: 200px; max-width: 260px; } }
</style>

<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">$links_html</div>
  </div>
</div>

<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";

  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }

  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      var willCollapse = !box.classList.contains("sat-about-collapsed");
      setCollapsed(box, willCollapse);
    });

    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }

  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_code = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        issue_number=issue_number,
        issue_url=issue_url,
        map_name=map_name,
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
    )

    m.get_root().html.add_child(folium.Element(html_code))


# =====================
# 1. Hämta SAT-etapper från Wikidata
# =====================
print("🔍 Hämtar SAT-etapper från Wikidata…")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(
    """
    SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
      ?item wdt:P361 wd:Q131318799;
            wdt:P31 wd:Q2143825;
            wdt:P402 ?osmid.
      OPTIONAL { ?item wdt:P706 ?island. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    """
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append(
        {
            "id": r["osmid"]["value"],
            "label": r.get("itemLabel", {}).get("value", ""),
            "island": r.get("islandLabel", {}).get("value", ""),
        }
    )

# =====================
# 2. Hämta geometrier från Overpass
# =====================
print(f"📡 Hämtar geometrier för {len(etapper)} OSM-relationer…")
overpass_url = "http://overpass-api.de/api/interpreter"
geom_per_rel = defaultdict(list)
all_lines = []

for e in etapper:
    rel_id = e["id"]
    query = f"""
    [out:json];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    r = requests.post(overpass_url, data={"data": query})
    if r.status_code != 200:
        print(f"⚠️ Fel för relation {rel_id}")
        continue
    data = r.json()
    for el in data.get("elements", []):
        if el.get("type") == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            if len(coords) >= 2:
                geom = LineString(coords)
                geom_per_rel[rel_id].append(geom)
                all_lines.append(geom)

if not all_lines:
    raise SystemExit("❌ Inga geometrier hämtades – kontrollera nätverk/Overpass.")

gdf_trail = gpd.GeoDataFrame(geometry=all_lines, crs="EPSG:4326")

# Samla varje relations geometrier som MultiLineString
geometries = [
    MultiLineString(geom_per_rel[e["id"]])
    if len(geom_per_rel[e["id"]]) > 1
    else geom_per_rel[e["id"]][0]
    for e in etapper if geom_per_rel[e["id"]]
]
meta_gdf = gpd.GeoDataFrame(
    [e for e in etapper if geom_per_rel[e["id"]]], geometry=geometries, crs="EPSG:4326"
)

# =====================
# 3. Skapa buffertar (50 m, 100 m, 200 m)
# =====================
print("🧮 Skapar buffertar 50/100/200 m…")
trail_utm = gdf_trail.to_crs(3006)

buffer50_utm = trail_utm.buffer(50)
buffer100_utm = trail_utm.buffer(100)
buffer200_utm = trail_utm.buffer(200)

buffer50 = buffer50_utm.to_crs(4326)
buffer100 = buffer100_utm.to_crs(4326)
buffer200 = buffer200_utm.to_crs(4326)

# =====================
# 4. Hämta toaletter från Overpass inom bbox
# =====================
bbox = gdf_trail.total_bounds
query_toilets = f"""
[out:json][timeout:25];
node["amenity"="toilets"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out;
"""
print("🚽 Hämtar toaletter från Overpass API…")
r = requests.post(overpass_url, data={"data": query_toilets})
if r.status_code != 200:
    raise SystemExit("❌ Kunde inte hämta toaletter – kontrollera nätverk/Overpass.")

toilets_data = r.json()

toilets = []
for el in toilets_data.get("elements", []):
    if el.get("type") == "node":
        pt = Point(el["lon"], el["lat"])
        toilets.append({"geometry": pt, "tags": el.get("tags", {}), "id": el["id"]})

gdf_toilets = gpd.GeoDataFrame(toilets, crs="EPSG:4326")
print(f"✅ Hittade {len(gdf_toilets)} toaletter inom bbox")

# =====================
# 5. Filtrera toaletter inom 200 m (kan ändras)
# =====================
in_range200 = gdf_toilets[gdf_toilets.geometry.within(buffer200.union_all())]
print(f"✅ {len(in_range200)} toaletter ligger inom 200 m från leden")

# =====================
# 6. Närmaste etapp för metadata i popup
# =====================
print("🔗 Matchar toaletter till närmaste etapp…")

toilets_utm = in_range200.to_crs(3006)
meta_utm = meta_gdf.to_crs(3006)
joined = gpd.sjoin_nearest(
    toilets_utm, meta_utm[["label", "island", "geometry"]], how="left", distance_col="distance_m"
).to_crs(4326)

# =====================
# 7. Bygg interaktiv karta
# =====================
print("🗺️ Bygger karta…")

# Starta vid centroid av första geometri
center = [gdf_trail.geometry.iloc[0].centroid.y, gdf_trail.geometry.iloc[0].centroid.x]
m = folium.Map(location=center, zoom_start=9)

# a) Etappgeometri – ett enda lager
folium.GeoJson(
    meta_gdf.union_all(),
    name="SAT-leden",
    tooltip="SAT-leden",
    style_function=lambda x: {"color": "blue", "weight": 3},
).add_to(m)

# b) Buffertlager (50/100/200 m) – färgkodade
folium.GeoJson(
    buffer50.union_all(),
    name="Buffert 50 m",
    style_function=lambda x: {"fillColor": "#2ECC71", "color": "#2ECC71", "weight": 1, "fillOpacity": 0.12},
).add_to(m)

folium.GeoJson(
    buffer100.union_all(),
    name="Buffert 100 m",
    style_function=lambda x: {"fillColor": "#F39C12", "color": "#F39C12", "weight": 1, "fillOpacity": 0.12},
).add_to(m)

folium.GeoJson(
    buffer200.union_all(),
    name="Buffert 200 m",
    style_function=lambda x: {"fillColor": "#E74C3C", "color": "#E74C3C", "weight": 1, "fillOpacity": 0.12},
).add_to(m)

# c) Toalettmarkörer (inkl. särskilt lager för wheelchair=yes)
wheel_group = FeatureGroup(name="Toaletter: wheelchair=yes (inom 200 m)")
good_group = FeatureGroup(name="Godkända toaletter (inom 200 m)")
warn_group = FeatureGroup(name="Varningar / Ofullständiga (inom 200 m)")

for _, row in joined.iterrows():
    tags = row["tags"]
    el_id = row["id"]
    osm_url = f"https://www.openstreetmap.org/node/{el_id}"
    wheelmap_url = f"https://wheelmap.org/node/{el_id}"

    popup_text = (
        f"<b><a href='{osm_url}' target='_blank'>OSM objekt</a></b> · "
        f"<a href='{wheelmap_url}' target='_blank'>Wheelmap</a><br>"
    )

    # Metadata
    if "image" in tags:
        popup_text += f"<img src='{tags['image']}' width='200'><br>"
    if "opening_hours" in tags:
        popup_text += f"<b>Öppettider:</b> {tags['opening_hours']}<br>"
    if "wheelchair" in tags:
        popup_text += f"<b>Rullstol:</b> {tags['wheelchair']}<br>"
    if "toilets:disposal" in tags:
        popup_text += f"<b>Avfallshantering:</b> {tags['toilets:disposal']}<br>"

    popup_text += "<hr style='margin:5px 0;'><ul style='padding-left:18px;'>"

    required = ["amenity", "access", "unisex", "toilets:disposal", "wheelchair"]
    recommended = [
        "toilets:handwashing",
        "toilets:paper_supply",
        "opening_hours",
        "fee",
        "operator",
        "description",
    ]
    missing_req = [t for t in required if t not in tags]
    missing_opt = [t for t in recommended if t not in tags]

    for t in missing_req:
        popup_text += f"<li><b>{t}</b> saknas</li>"
    for t in missing_opt:
        popup_text += f"<li>{t} saknas</li>"
    popup_text += "</ul><i>Tips: kontrollera viktiga attribut</i>"

    # Lagerlogik
    is_wheel_yes = tags.get("wheelchair") == "yes"
    if is_wheel_yes:
        # Särskild ikon/lager för wheelchair=yes
        Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=Popup(popup_text, max_width=300),
            icon=Icon(color="green", icon="wheelchair", prefix="fa"),
        ).add_to(wheel_group)
        continue

    # Övriga markörer i godkända/varningar
    icon_color = "red" if missing_req else "blue"
    Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=Popup(popup_text, max_width=300),
        icon=Icon(color=icon_color, icon="info-sign"),
    ).add_to(warn_group if missing_req else good_group)

m.add_child(wheel_group)
m.add_child(good_group)
m.add_child(warn_group)
LayerControl(collapsed=False).add_to(m)

# =====================
# 8. Spara
# =====================
add_about_box(m, issue_number=132, map_name="Toaletter nära leden")
save_map_multiple(m)


🔍 Hämtar SAT-etapper från Wikidata…
📡 Hämtar geometrier för 20 OSM-relationer…
🧮 Skapar buffertar 50/100/200 m…
🚽 Hämtar toaletter från Overpass API…
✅ Hittade 602 toaletter inom bbox
✅ 80 toaletter ligger inom 200 m från leden
🔗 Matchar toaletter till närmaste etapp…
🗺️ Bygger karta…


/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_36335/1262749122.py:317: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  in_range200 = gdf_toilets[gdf_toilets.geometry.within(buffer200.unary_union)]
/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_36335/1262749122.py:342: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  meta_gdf.unary_union,
/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_36335/1262749122.py:350: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  buffer50.unary_union,
/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_36335/1262749122.py:356: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  buffer100.unary_union,
/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_36335/1262749122.py:362: DeprecationWarning: The

✅ Karta sparad:
  • kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_21.html
  • kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_latest.html
  • kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_09_20_19_15.html


/Users/salgo/Documents/GitHub/Stockholm_Archipelago_Trail/notebook


In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

NameError: name 'time' is not defined